<a href="https://colab.research.google.com/github/ShyamSundhar1411/My-ML-Notebooks/blob/master/ML%20Algorithms/Decision_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Decision Tree

In [1]:
import numpy as np
from collections import Counter

In [107]:
class Node:
  def __init__(self,feature=None,threshold = None,left = None,right = None,*,value = None):
    self.feature = feature
    self.th = threshold
    self.left = left
    self.right = right
    self.value = value
  def is_leaf_node(self):
    return self.value is not None

In [108]:
class DecisionTree:
  def __init__(self,sample_split = 2,max_depth = 100,n_features = None):
    self.sample_split = sample_split
    self.depth = max_depth
    self.n_features = n_features
    self.root = None

  def fit(self,X,Y):
    self.n_features = X.shape[1] if not self.n_features  else min(X.shape[1],self.n_features)
    self.root = self.grow(X,Y)
    return self

  def traverse(self,x,node):
    if node.is_leaf_node():
      return node.value
    if x[node.feature]<=node.th:
      return self.traverse(x,node.left)
    return self.traverse(x,node.right)

  def predict(self,X):
    return np.array([self.traverse(x,self.root) for x in X])
  
  def calculate_leaf_value(self,y):
    return Counter(y).most_common(1)[0][0]


  def calculate_entropy(self,y):
    count = np.bincount(y)
    p_x = count/len(y)
    return -np.sum([p*np.log(p) for p in p_x if p>0])



  def split(self,x_col,threshold):
    left_index = np.argwhere(x_col<=threshold).flatten()
    right_index = np.argwhere(x_col>threshold).flatten()
    return left_index,right_index


  def calculate_information_gain(self,y,X_col,threshold):
    parent_entropy = self.calculate_entropy(y)

    left_index,right_index = self.split(X_col,threshold)
    if len(left_index) == 0 or len(right_index) == 0:
      return 0
    n = len(y)
    n_left = len(left_index)
    n_right = len(right_index)
    entropy_left = self.calculate_entropy(y[left_index])
    entropy_right = self.calculate_entropy(y[right_index])
    child_entropy = (n_left/n)*entropy_left+(n_right/n)*entropy_right
    return parent_entropy-child_entropy

    

  #Randomness in Decision Tree
  def best_split(self,X,Y,feature_idx):
    best_gain = -1
    split_index,split_thresgold = None,None
    for index in feature_idx:
      x_col = X[:,index]
      thresholds = np.unique(x_col)

      for th in thresholds:
        #Information Gain
        gain = self.calculate_information_gain(y,x_col,th)
        if gain > best_gain:
          best_gain = gain
          split_index = index
          split_threshold = th
    return split_index,split_threshold


  def grow(self,X,Y,depth=0):
    n_samples,n_features = X.shape
    n_labels = len(np.unique(Y))

    if (depth>=self.depth or n_labels==1 or n_samples < self.sample_split):
      return Node(value=self.calculate_leaf_value(Y))
    feature_index = np.random.choice(n_features,self.n_features,replace = False)
    best_feature,best_th = self.best_split(X,Y,feature_index)
    left_index,right_index = self.split(X[:,best_feature],best_th)
    left = self.grow(X[left_index,:],Y[left_index],depth+1)
    right = self.grow(X[right_index,:],Y[right_index],depth+1)
    return Node(best_feature,best_th,left,right)
    
  def accuracy(self,y_pred,y_true):
    return np.sum(y_pred==y_true)/len(y_pred)


In [109]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [110]:
data = datasets.load_breast_cancer()
x,y = data.data,data.target

In [111]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [112]:
model = DecisionTree()
model.fit(x_train,y_train)

In [113]:
y_pred = model.predict(x_test)

In [114]:
y_pred

array([1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0])

In [115]:
model.accuracy(y_pred,y_test)

0.8421052631578947